In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import json
import datetime
from datetime import datetime
from pandas import read_excel

In [2]:
########## Saving json file from link ##########

from urllib.request import urlopen

url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
with urlopen(url) as source:
    out = source.read()

data = json.loads(out)

with open("restaurant_data.json", "w") as file:
    merged = []
    for i in range(len(data)):
        # combine all results shown
        merged.extend(data[i]["restaurants"])
    json.dump(merged, file)

In [3]:
mongodb = MongoClient('localhost', 27017)
mongodb.drop_database('Restaurants')

In [4]:
########## Load data files & save as them collections of a database in mongodb ##########

db = mongodb.Restaurants

restaurant_data = json.load(open("restaurant_data.json"))
# create and populate collection
res = db.restaurants
res.insert_many(restaurant_data)

country_code = read_excel("Country-Code.xlsx")
country_code = country_code.to_dict(orient='records')
# create and populate collection
ccode = db.countrycode
ccode.insert_many(country_code)

## 1. Generating restaurants.csv
Extract fields:
* Restaurant Id
* Restaurant Name
* Country
* City
* User Rating Votes
* User Aggregate Rating (float)
* Cuisines

In [5]:
out = res.aggregate([ 
    {"$group": {
        "_id": "$restaurant.R.res_id", "Restaurant Name": {"$first":"$restaurant.name"}, 
        "Country": {"$first":"$restaurant.location.country_id"}, 
        "City": {"$first":"$restaurant.location.city"}, 
        "User Rating Votes": {"$first":"$restaurant.user_rating.votes"}, 
        "User Aggregate Rating": {"$first":"$restaurant.user_rating.aggregate_rating"}, 
        "Cuisines": {"$first":"$restaurant.cuisines" }
    }}
])
df1 = pd.DataFrame(list(out))
df1.rename(columns={"_id":"Restaurant Id"}, inplace=True)

# define function that maps country_id to country
def get_Country_from_CountryCode(country_id, collection):
    out = collection.find_one({"Country Code":country_id}, {"_id":0, "Country":country_id})
    if not out:
        return "NA"
    else:
        return out["Country"]

# map country_id to country
df1["Country"] = df1["Country"].apply(lambda country_id: get_Country_from_CountryCode(country_id, ccode))
# change dtype of User Aggregate Rating to float
df1["User Aggregate Rating"] = pd.to_numeric(df1["User Aggregate Rating"])

In [6]:
df1

,Restaurant Id,Restaurant Name,Country,City,User Rating Votes,User Aggregate Rating,Cuisines
0,2300729,Juicebox,India,Kanpur,338,4.3,"Beverages, Fast Food"
1,18748677,The Civil Kitchen,India,Nagpur,232,4.2,"Chinese, Sandwich, Fast Food, North Indian, Bu..."
2,901830,Mocha Art Cafe,India,Kochi,602,4.7,"Cafe, Continental, European, Italian, Seafood"
3,5702990,Jazz@PizzaExpress,UAE,Abu Dhabi,1374,4.8,"Italian, Pizza, Salad"
4,18699247,Dining Room,India,Ooty,8,3.4,"Armenian, Chinese, European, Italian, North In..."
...,...,...,...,...,...,...,...
1295,18312984,Sahib’s Barbeque by Ohri’s,India,Hyderabad,5346,4.7,"Hyderabadi, Awadhi, BBQ"
1296,18767581,Saint,South Africa,Sandton,173,3.9,Italian
1297,18770281,Three Kings,India,Bhubaneshwar,333,4.5,"Arabian, Mughlai, Lebanese"
1298,1401685,Dutt Guru Kripa,India,Indore,1138,4.0,"North Indian, Fast Food"


In [7]:
# save as csv locally
df1.to_csv("restaurants.csv")

## 2. Generating restaurant_events.csv
Extract fields (start <= April 2019 <= end):
* Event Id
* Restaurant Id
* Restaurant Name
* Photo URL
* Event Title
* Event Start Date
* Event End Date

In [8]:
column_names = ["Event Id", "Restaurant Id", "Restaurant Name",
                "Photo URL", "Event Title", "Event Start Date", "Event End Date"]

events_dict = dict(zip(column_names, [[] for i in range(len(column_names))]))

# filter restaurants with events that happened in April 2019
out = res.find({
    "$and":[
        {"restaurant.zomato_events.event.start_date":{"$lt":'2019-05'}}, 
        {"restaurant.zomato_events.event.end_date":{"$gte":'2019-04'}}
    ]
}, {"_id":0, "restaurant.R.res_id":1, "restaurant.name":1, "restaurant.zomato_events.event.event_id":1, 
"restaurant.zomato_events.event.photos":1, "restaurant.zomato_events.event.title":1, 
"restaurant.zomato_events.event.start_date":1, "restaurant.zomato_events.event.end_date":1})

for doc in out:
    # doc is one restaurant
    doc = doc["restaurant"]
    # restaurant.zomato_events conrtains a list of events
    for event in doc["zomato_events"]:
        event = event["event"]
        if event["start_date"] >= '2019-05' or event["end_date"] < '2019-04':
            continue
        events_dict["Event Id"].append(event["event_id"])
        events_dict["Event Title"].append(event["title"])
        events_dict["Restaurant Id"].append(doc["R"]["res_id"])
        events_dict["Restaurant Name"].append(doc["name"])
        # take the url of the first photo if any
        photo_urls = [p["photo"]["url"] for p in event["photos"]]
        if not photo_urls:
            events_dict["Photo URL"].append("NA")
        else:
            events_dict["Photo URL"].append(photo_urls[0])
        events_dict["Event Start Date"].append(event["start_date"])
        events_dict["Event End Date"].append(event["end_date"])

In [9]:
df2 = pd.DataFrame(events_dict)

In [10]:
df2

,Event Id,Restaurant Id,Restaurant Name,Photo URL,Event Title,Event Start Date,Event End Date
0,322331,18649486,The Drunken Botanist,https://b.zmtcdn.com/data/zomato_events/photos...,BackToBasic Wednesdays !!\n\n\n,2019-03-06,2019-08-28
1,332812,308322,Hauz Khas Social,https://b.zmtcdn.com/data/zomato_events/photos...,Live 20/20 Match Screenings,2019-03-29,2019-05-23
2,336644,18856789,AIR- An Ivory Region,https://b.zmtcdn.com/data/zomato_events/photos...,Dhol Bhangra Night,2019-04-10,2019-04-11
3,336889,18382360,Local,NA,Cocktail Wednesday,2019-04-10,2019-04-10
4,336888,18382360,Local,NA,Cocktail Wednesday,2019-04-10,2019-04-10
...,...,...,...,...,...,...,...
177,305850,7001670,"Eight - Cordis, Auckland Hotel",https://b.zmtcdn.com/data/zomato_events/photos...,Smoky Thursdays,2019-03-01,2019-04-30
178,298162,7001670,"Eight - Cordis, Auckland Hotel",https://b.zmtcdn.com/data/zomato_events/photos...,Chocolate Tuesdays,2019-04-05,2019-08-21
179,329300,18981544,Katsu YA,https://b.zmtcdn.com/data/zomato_events/photos...,Review us on Zomato,2019-03-22,2019-04-30
180,329301,18981544,Katsu YA,https://b.zmtcdn.com/data/zomato_events/photos...,Opening Promo,2019-03-22,2019-04-30


In [11]:
# some checking
list(res.find({
    "$or":[
        {"restaurant.zomato_events.event.start_date":{"$gte":'2019-05'}}, 
        {"restaurant.zomato_events.event.end_date":{"$lt":'2019-04'}}
    ]
}, {"_id":0, "restaurant.R.res_id":1, "restaurant.name":1, "restaurant.zomato_events.event.event_id":1, 
"restaurant.zomato_events.event.photos":1, "restaurant.zomato_events.event.title":1, 
"restaurant.zomato_events.event.start_date":1, "restaurant.zomato_events.event.end_date":1}))

[{'restaurant': {'R': {'res_id': 18560275},
   'name': 'Barbeque Nation',
   'zomato_events': [{'event': {'event_id': 331382,
      'start_date': '2019-05-05',
      'end_date': '2019-06-06',
      'photos': [],
      'title': 'Ramadan '}}]}},
 {'restaurant': {'R': {'res_id': 5702486},
   'name': 'Ghee Rice',
   'zomato_events': [{'event': {'event_id': 335542,
      'start_date': '2019-04-14',
      'end_date': '2019-04-14',
      'photos': [],
      'title': 'Vishu Celebration at Ghee Rice'}},
    {'event': {'event_id': 332697,
      'start_date': '2019-05-04',
      'end_date': '2019-06-04',
      'photos': [],
      'title': 'Grand Iftar Buffet 2019'}}]}}]

In [12]:
df2.loc[df2["Restaurant Id"]==5702486]

,Event Id,Restaurant Id,Restaurant Name,Photo URL,Event Title,Event Start Date,Event End Date
174,335542,5702486,Ghee Rice,NA,Vishu Celebration at Ghee Rice,2019-04-14,2019-04-14


In [13]:
df2.to_csv("restaurant_events.csv")

## 3. Determine threshold for different rating text based on aggregate rating.
Return aggregates for the following ratings only:
* Excellent
* Very Good
* Good
* Average
* Poor

In [14]:
ratings = ["Excellent", "Very Good", "Good", "Average", "Poor"]

out = res.aggregate([
    {"$match": {
        "restaurant.user_rating.rating_text": {"$in":ratings}
    }},
    {"$group":{
        "_id": "$restaurant.user_rating.rating_text",
        "min_rating": {"$min": "$restaurant.user_rating.aggregate_rating"},
        "max_rating": {"$max": "$restaurant.user_rating.aggregate_rating"}
    }}
])

In [15]:
pd.DataFrame(list(out))

,_id,min_rating,max_rating
0,Very Good,4.0,4.4
1,Excellent,4.5,4.9
2,Average,2.5,3.4
3,Poor,2.2,2.2
4,Good,3.5,3.9
